# Assessing equilibration in MD simulations

### A notebook to accompany the CCP-BioSim training workshop "Trusting Your Simulations".

## Introduction

If we plot the temperature of a MD simulation as it progresses we will almost always see a graph of this general form:

![fig](equilt.png)

We will likely get the same sort of plot for other physical metrics - e.g. density. 

Our model for how molecular dynamics simulations progress from their point of initialization is heavily influenced by this: that there is first an equilibration phase, and then a "production phase". If we wish to model the behaviour of an equilibrated system, then only data from the production phase is relevant. In the example shown here, we might decide that the system is equilibrated after about 300-400 snapshots.

The problem with this approach is that depending on the metric we use, the point at which a simulation seems equilibrated can appear to vary. For example, equilibration of temperature may seem to occur faster than equilibration of density. Which metric is most relevant? 

A common choice in MD work is to look at the root mean square deviation of the cooordinates (RMSD) from the starting structure. Plots of RMSD against time very often look similar to the one above, though the time to reach a stable value is typically much longer than for temperature or pressure. 

Whatever metric is chosen, a second, more subtle, aspect of equilibration that this model encourages is that it is something that can be assessed **before** the data collection phase begins.

The aim of this notebook is to:

* Illustrate that RMSD is generally not a good metric to use to assess equilibration
* Illustrate that thinking of equilibration as something that can be assessed independently of the production phase is not the best approach.

----------

The data comes from a 4 microsecond simulation of chignolin: a small peptide that oscillates between a folded and unfolded state. The simulation comes from a tutorial created by Lillian Chong's group at the University of Pittsburgh for their advanced sampling software [WESTPA](https://westpa.github.io/westpa/) - please checkout this package, it's very useful and well documented!

In ther first part of the notebook we explore the conventional approach: we imagine we just have the data from the first 300 nanoseconds of the simulation (1500 snapshots), and see what RMSD analysis suggests the equilibration time is.

In the second part of the notebook we explore a more "holistic" approach to analysing equilibration and sampling.

----

Begin by loading the required Python packages: numpy for the maths, mdtraj for trajectory analysis, matplotlib for the graphs:

In [ ]:
import numpy as np
import mdtraj as mdt
from matplotlib import pyplot as plt
%matplotlib inline

## Part 1: Estimating equilibration from RMSD plots

Load the first 1500 frames of the trajectory file, calculate the rmsd of each snapshot from the first, and plot:

In [ ]:
t = mdt.load('chignolin.xtc', top='chignolin.pdb')[:1500]
r = mdt.rmsd(t, t[0])
plt.plot(r)
plt.xlabel('snapshot number')
plt.ylabel('rmsd (nanometers)')

It looks like there is some sort of equilibration after 700 snapshots, rmsd stabilizing around 0.4-0.5 nm. Recalculate the rmsd using snapshot 700 as the starting point and reference:

In [ ]:
r = mdt.rmsd(t[700:], t[700])
plt.plot(range(700, 1500), r)
plt.xlabel('snapshot number')
plt.ylabel('rmsd (nanometers)')

Oh - now it looks like there is some sort of equilibration after 1000 snapshots, rmsd stabilizing around 0.4-0.5 nm. Try again, recalculate the rmsd using snapshot 1000 as the starting point and reference:

In [ ]:
r = mdt.rmsd(t[1000:], t[1000])
plt.plot(range(1000, 1500), r)
plt.xlabel('snapshot number')
plt.ylabel('rmsd (nanometers)')

Oh - now it looks like there is some sort of equilibration after 1100 snapshots, rmsd stabilizing around 0.4-0.5 nm. Try again, recalculate the rmsd using snapshot 1100 as the starting point and reference:

In [ ]:
r = mdt.rmsd(t[1100:], t[1100])
plt.plot(range(1100, 1500), r)
plt.xlabel('snapshot number')
plt.ylabel('rmsd (nanometers)')

It seems we could go on for ever like this - chopping off more and more of the initial data, but never quite getting to what looks like a fully equilibrated system.

**However...**

Instead of just loooking at the first 300ns (1500 snapshots), let's look at the full 4 microseconds of data:

In [ ]:
t = mdt.load('chignolin.xtc', top='chignolin.pdb')
r = mdt.rmsd(t, t[0])
plt.plot(r)
plt.xlabel('snapshot number')
plt.ylabel('rmsd (nanometers)')

We have a problem! The idea that the RMSD is going to stablize to a more-or-less constant value representative of equilibrium is clearly wrong. How do we deal with this situation? In part 2 of the notebook we look at one possible approach.

## Part 2: A holistic approach to the estimation of equilibration

The RMSD plot above actually makes sense: over the MD the peptide is 'flipping' between a folded state (rmsd < 0.35 nanometers) and unfolded state(s) (rmsd > 0.35 nanometers). There is also, presumably, some form of equilibration going on as well though - but where is it?

The first thing is to decide what the metric of interest is - what ultimately was the purpose of doing this simulation? Let's assume it was to make a prediction of the fraction of the time the peptide adopts a folded state, `f_folded`. Taking all the data, calculate `f_folded`:

In [ ]:
f_folded = (r < 0.35).sum() / len(r)
print('f_folded = {}'.format(f_folded))

How well converged is this value? Calculate `f_folded` for the first and second halves of the simulation independently, and compare:

In [ ]:
half_data = len(r) // 2
ff1 = (r[:half_data] < 0.35).sum()/half_data
ff2 = (r[half_data:] < 0.35).sum()/half_data
print('f_folded from 1st half of simulation: {}'.format(ff1))
print('f_folded from 2nd half of simulation: {}'.format(ff2))

The two estimates are very different from each other. Maybe this is because the first half of the complete simulation includes an equilibration phase, for which the data should not be included.

The code below trims off larger and larger sections from the start of the trajectory then calculates `f_folded` for the first and second halves of what data remains. From these it calculates the mean and unsigned difference. Finally the data is plotted.

In [ ]:
delta_f_folded = []
mean_f_folded = []
# trim off up to 10,000 snapshots (half of all the data) in 100 snapshot increments:
discarded_snapshots = range(0, 10000, 100)
for neq in discarded_snapshots: 
    req = r[neq:]
    l = len(req) // 2
    ff1 = (req[:l] < 0.35).sum() / l
    ff2 = (req[l:] < 0.35).sum() / l
    delta_f_folded.append(np.abs(ff1 - ff2))
    mean_f_folded.append((ff1 + ff2) / 2)

plt.plot(discarded_snapshots, delta_f_folded, label='difference')
plt.plot(discarded_snapshots, mean_f_folded, label='mean')
plt.xlabel('number of discarded snapshots')
plt.ylabel('f_folded')
plt.legend()

The graph shows clearly that by discarding increasing amounts of the initial data, up to about 3000 snapshots, the values of `f_folded` calculated from the first and second halves of the remaining data become increasingly close to each other (the difference falls steadily towards zero). This suggests that we are trimming off more and more of the unrepresentative equilibration phase. After 3000 snapshots though the trend is - somewhat erratically - reversed, the difference begins to increase. This makes sense: as the number of snapshots in each sub-sample is decreased, random fluctuations make it less likely the two averages will be the same. 

Since it's never a good idea to discard any more data than you have to, let's settle on 3000 snaphots (representing the first 600 nanoseconds) as being the equilibration time and recalculate `f_folded` from what remains:

In [ ]:
req = r[3000:]
f_folded = (req < 0.35).sum() / len(req)
print('f_folded = {}'.format(f_folded))

Note this value is more than 10% higher than what was calculated using all the simulation data.

## Conclusion:

Two take-home messages could be:

* Calculate equilibration on the basis of the actual metric you care about.
* Be holistic: evaluate equilibration and sampling/reproducibility at the same time.

-----

## Exercise 1 (easier):

Imagine that we have a different metric of interest: the proportion of the time that the end-to-end distance of the peptide is less than 0.3 nanometers. A Python expression to calculate this would be:

    r = mdt.compute_distances(t, [[0, t.n_atoms -1]])
    
Adapt the code above to estimate the equilibrium time and best estimate of this metric in this scenario.

## Exercise 2 (harder):

From the timeseries graph you will see that it could be more accurate to consider the dynamics of chignolin as involving transitions between three states, not two: a 'folded' one (rmsd < 0.35), an 'intermediate' one (0.35 < rmsd < 0.6) and an 'extended' one (rmsd > 0.6).

Create a Python function that, given a series of rmsd values, will calculate the fractions of conformations in each of these three states `(f_folded, f_intermediate, f_extended)`, and a second Python function that, given a pair of distributions `(f_folded1, f_intermediate1, f_extended1)` and `(f_folded2, f_intermediate2, f_extended2)`, returns a similarity metric (zero if they are identical, more positive the more different they are). Then adapt the code above to estimate the equilibrium time and best estimates of these metrics in this scenario.

*Charlie Laughton September 2020*